<a href="https://colab.research.google.com/github/Vac1k/Aurora_Project/blob/main/Aurora_Predict_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпорт бібліотек

In [38]:
import pandas as pd
import speech_recognition as sr
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from docx import Document
import gensim.downloader as api
import numpy as np

In [ ]:
!pip install SpeechRecognition
!pip install python-docx

Завантаження та обробка даних про товари

In [39]:
file_path_inventory = "/content/Залишки по ТТ.xlsx"
inventory_df = pd.read_excel(file_path_inventory)

In [40]:
inventory_df = inventory_df.rename(columns={
    "Рівень категорії 3": "Category_Level3",
    "Назва товару": "Product_Full_Name",
    "К-сть залишків на кінець дня": "Stock_Quantity_End_Day",
    "Собівартість залишків на кінець дня": "Stock_Cost_End_Day"
})

In [41]:
inventory_df["Product_ID"] = inventory_df["Product_Full_Name"].str.extract(r'^(\d+)')
inventory_df["Product_Name"] = inventory_df["Product_Full_Name"].str.replace(r'^\d+\s+', '', regex=True)
inventory_df["Product_ID"] = pd.to_numeric(inventory_df["Product_ID"], errors='coerce')
inventory_df["Calculated_Price"] = (inventory_df["Stock_Cost_End_Day"] / inventory_df["Stock_Quantity_End_Day"]).round(3)

Завантаження подій та їх аналіз

In [42]:
def load_events_from_docx():
    doc = Document("/content/12123.docx")
    events_dict, current_event = {}, None

    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()
        if not text:
            continue
        if text[0].isdigit() and "." in text:
            current_event = text.split(". ", 1)[1]
            events_dict[current_event] = []
        elif current_event:
            events_dict[current_event].append(text)
    return events_dict

events_data = load_events_from_docx()

Векторизація товарів для AI-пошуку

In [43]:
ai_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
inventory_df["Product_Vector"] = inventory_df["Product_Name"].apply(lambda x: ai_model.encode(x))

word2vec_model = api.load("word2vec-ruscorpora-300")

Фільтрація товарів за подією

In [44]:
def find_similar_words(word, top_n=3):
    try:
        return [w[0] for w in word2vec_model.most_similar(word, topn=top_n)]
    except KeyError:
        return []

def filter_by_event(event_name):
    allowed_keywords = events_data.get(event_name, [])
    if not allowed_keywords:
        return pd.DataFrame(columns=inventory_df.columns)

    filtered_df = inventory_df[inventory_df["Product_Name"].apply(
        lambda x: any(word.lower() in x.lower() for word in allowed_keywords)
    )]

    if filtered_df.empty:
        return find_similar_products(event_name, top_n=5)

    return filtered_df.head(5)

AI-пошук товарів

In [45]:
def find_similar_products(query, top_n=5):
    query_vector = np.array(ai_model.encode(query)).reshape(1, -1)
    product_vectors = np.array(list(inventory_df["Product_Vector"]))

    similarity_scores = cosine_similarity(query_vector, product_vectors).flatten()
    top_indices = similarity_scores.argsort()[-top_n:][::-1]

    return inventory_df.iloc[top_indices][["Product_Name", "Category_Level3", "Stock_Quantity_End_Day"]].head(top_n)

Speech Recognition

In [46]:
def recognize_speech():
    recognizer = sr.Recognizer()
    try:
        with sr.Microphone() as source:
            audio = recognizer.listen(source)
            return recognizer.recognize_google(audio, language="uk-UA")
    except (sr.UnknownValueError, sr.RequestError, OSError):
        return None

Основна логіка програми

In [47]:
def process_user_query(user_input):
    user_input = user_input.lower().strip()
    matched_event = next((event for event in events_data if event.lower() in user_input), None)

    if matched_event:
        return filter_by_event(matched_event)

    expanded_keywords = user_input.split()
    for word in user_input.split():
        expanded_keywords.extend(find_similar_words(word))

    return find_similar_products(" ".join(expanded_keywords), top_n=5)

In [48]:
def main():
    mode = input("🔹 Виберіть режим (1 - текстовий ввод, 2 - голосовий ввод): ").strip()
    user_query = input("✍ Введіть ваш запит: ") if mode == "1" else recognize_speech()
    if not user_query:
        return

    recommendations = process_user_query(user_query)
    print(f"📌 Рекомендації для запиту '{user_query}':")
    print(recommendations.to_string(index=False))

In [64]:
main()

🔹 Виберіть режим (1 - текстовий ввод, 2 - голосовий ввод): 1
✍ Введіть ваш запит: я хочу подарувати щось батьку для рибалки
📌 Рекомендації для запиту 'я хочу подарувати щось батьку для рибалки':
                                         Product_Name          Category_Level3  Stock_Quantity_End_Day
Снасть риболовна фідерна для ловлі коропа "Fisherman"   29.6 рибальські снасті                       6
   Снасть риболовна годівниця з відводами "Fisherman"   29.6 рибальські снасті                       8
     Набір повідців для риболовлі Fisherman арт.50531 29.9 рибальське приладдя                      11
         Снасть риболовна фідерна Fisherman арт.50488   29.6 рибальські снасті                      11
                        Прикормка риболовна Best Аніс  29.2 приманки,прикормки                       6
